# Tavily Web Search

In [1]:
%pip install -qU langchain-tavily


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(
    max_results=5,
    topic="general",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [4]:
search_tool.invoke({'query': '전세계에서 가장 많이 쓰이는 API를 설명해줘'})

{'query': '전세계에서 가장 많이 쓰이는 API를 설명해줘',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.reddit.com/r/webdev/comments/ydrh3t/most_common_apis/?tl=ko',
   'title': '가장 흔한 API들 : r/webdev - Reddit',
   'content': 'Missing: 전세계 해줘',
   'score': 0.7274535,
   'raw_content': None},
  {'url': 'https://velog.io/@sae0428/REST-API-RESTful-API',
   'title': 'REST API & RESTful API - velog',
   'content': 'REST API 란 이전 포스트에서 작성한 REST의 원리를 따르는 API를 의미한다. ... 전 세계에서 가장많이 쓰이는 JSON 형태로 주고받는 API. REST API는 모든',
   'score': 0.6698534,
   'raw_content': None},
  {'url': 'https://m.blog.naver.com/codingbarbie/223233443109',
   'title': 'API(Application Programming Interface)란 무엇인가? / API 종류 ...',
   'content': '대표적으로 많이 사용하는 API. Google Maps API: 지도 서비스를 애플리케이션에 통합. Twitter API: 트위터의 여러 기능을 애플리케이션에 통합.',
   'score': 0.49362883,
   'raw_content': None},
  {'url': 'https://blog.naver.com/inpink_/223498702649',
   'title': '4가지 유명한 API 프로토콜: REST, GraphQL, Webh

In [5]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model='gpt-4.1-nano') # llm 인스턴스 생성

prompt = PromptTemplate(
    input_variables=['query', 'search_results'],
    template="""넌 웹 검색결과를 요약해서 사용자의 질문에 맞게 좋은 답을 해주는 챗봇이야.

    질문: {query}
    검색결과: {search_results}
    질문에 맞는 답을 검색결과 바탕으로 간단하게 알려줘
    """
)

chain = (
    {
        'query': RunnablePassthrough(),  # 사용자 입력 그대로 이 자리에 들어옴
        'search_results': search_tool | RunnableLambda(
            lambda x: '\n'.join(
                [f'-{r['title']} ({r['content']})' for r in x['results']]
            )
        )
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('강남 신세계에서 점심 먹기 좋은곳')

'강남 신세계백화점에서 점심 식사 장소로는 타치바나(신세계백화점 지하 1층 하우스오브신세계 추천)가 인기 있으며, 다이닝코드와 트립어드바이저에서도 여러 맛집이 추천되고 있습니다. 또한, 미쉐린 빕 구르망 셰프의 점심 한정 반상도 허용되어 있으니 다양한 선택이 가능합니다.'

In [17]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from datetime import datetime

today = datetime.today().strftime('%D')

search_tool = TavilySearch(
    max_results=5,
    topic='general'
)

prompt = ChatPromptTemplate.from_messages([
    ('system', f'너는 훌륭한 어시스턴트야. 반드시 웹 검색도구를 사용해서 정보를 얻어. 최대한 답을 잘 해보자. 오늘은 {today}야.'),
    MessagesPlaceholder(variable_name='messages'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=[search_tool], verbose=True)

agent_executor.invoke({'messages': [{'role': 'human', 'content': '오늘 날씨에 맞는 음악 추천해줘'}]})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '오늘 날씨에 맞는 음악 추천', 'search_depth': 'advanced'}`


{'query': '오늘 날씨에 맞는 음악 추천', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.youtube.com/watch?v=f6X36lcmg6M', 'title': '나른한 요즘 날씨에 틀어놓기 좋은 산뜻하고 기분 좋은 노래 모음', 'content': "I'll say I love you twice [Music] [Music] thank you [Music] okay yes [Music] [Music] [Music] [Music] [Music] [Music] [Music] all right [Music] all right [Music] remember [Music] [Music] [Music] money [Music] right now [Music] in the morning [Music] follow me [Music] [Music] I should get a chance [Music] [Music] [Music] [Music] become true [Music] okay [Music] [...] [Music] your Channel [Music] [Music] Entertainer [Music] [Music] premium [Music] [Music] Harry Potter the Summers [Music] [Music] [Music] this morning [Music] [Music] [Music] [Music] [Music] [Music] um remember [Music] foreign [Music] [Music] [Music] I hear you [Music] thank you [Mus

{'messages': [{'role': 'human', 'content': '오늘 날씨에 맞는 음악 추천해줘'}],
 'output': '오늘 날씨는 대체로 흐림 또는 비 소식을 포함하고 있어 감성적인 분위기의 음악이 어울릴 것 같아요. 추천 음악으로는 감성적이고 잔잔한 분위기를 자아내는 노래들이 적합하겠습니다.\n\n1. **[비 오는 날 들으면 감성 터지는 노래 모음](https://www.youtube.com/watch?v=qPnPanVUsLU)** - 이 플레이리스트에는 헤이즈, 에픽하이 등 감성적인 노래들이 포함되어 있어 비 오는 날의 분위기와 잘 어울립니다.\n2. **잔잔한 음악 추천** - 부드럽고 기분 좋은 느낌의 곡들이 하루의 흐림과 어울릴 것 같습니다.\n\n이 외에도 음악을 들으며 차분한 시간 보내세요!'}